# Train a viseme classifier

> Train a viseme classifier in colab.

# TODO
- overview of pre-requisits

combine 0:no expression and 4:random into a single 0:ignore class

at inference time, we don't care about the difference

the class imbalance might also be useful - as this will be our do nothing default - and we only want to take action when we're sure a command is being requested

TODO: 
- save stats with model
- Try bigger batch size - even if it means we need more epochs
- model size/depth
- Exaggerate 0 imbalance - how far do we have to go to not got any 0s misclassified (on test set)
- see what happens if we don't use nose landmarks for training - try with/without normalizing for nose

In [ ]:
# run this, then restart the runtime
! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from fastai.tabular.all import *

In [6]:
from pathlib import Path
import numpy as np
import pandas as pd
import json

In [10]:
path = Path('data/viseme_dataset_20211113_114625/processed_20211113_145138')
# path = Path('/content/drive/MyDrive/Colab Notebooks/datasets/expoco/processed_20211019_205023')

In [11]:
print('loading from', path)
data = np.load(path/'data.npy')
print('data.shape', data.shape)
with open(path/'metadata.json') as f:
    metadata = json.load(f)
df = pd.DataFrame(data, columns=metadata['cont_names'] + [metadata['y_name']])
df

loading from data\viseme_dataset_20211113_114625\processed_20211113_145138
data.shape (2000, 213)


,5x,5y,2x,2y,218x,218y,438x,438y,0x,0y,...,408y,409x,409y,410x,410y,415x,415y,424x,424y,expression_id
0,0.505646,0.494332,0.503341,0.490903,0.498368,0.501670,0.506184,0.492390,0.507021,0.487174,...,0.466999,0.496324,0.469936,0.502941,0.464381,0.496418,0.444797,0.493698,0.493209,0.0
1,0.503841,0.494711,0.501086,0.488799,0.499251,0.501959,0.504125,0.492880,0.504448,0.487415,...,0.466239,0.493961,0.467831,0.499657,0.463201,0.489543,0.439547,0.495851,0.491819,0.0
2,0.496112,0.494247,0.497164,0.492464,0.500322,0.503856,0.495550,0.492506,0.494844,0.488023,...,0.476045,0.482862,0.479148,0.487478,0.472226,0.475334,0.449275,0.493696,0.497160,0.0
3,0.489471,0.498384,0.493200,0.493677,0.501554,0.505352,0.488306,0.496586,0.486446,0.491592,...,0.479880,0.472891,0.484327,0.478250,0.475697,0.458374,0.454175,0.489752,0.500201,0.0
4,0.487420,0.497972,0.490142,0.493929,0.499240,0.506324,0.486311,0.496072,0.484130,0.491305,...,0.483028,0.463874,0.487154,0.469231,0.478301,0.451558,0.452418,0.485923,0.501094,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,-2.283781,-2.233636,-2.284498,-2.249344,-2.286113,-2.278023,-2.283565,-2.230405,-2.283090,-2.227529,...,-2.224948,-2.249833,-2.229237,-2.260420,-2.224020,-2.249427,-2.208283,-2.269798,-2.259759,0.0
1996,-2.323838,-2.271298,-2.324637,-2.289553,-2.326421,-2.322912,-2.323605,-2.267545,-2.323097,-2.264216,...,-2.261390,-2.285296,-2.266389,-2.297457,-2.260306,-2.285031,-2.242138,-2.307770,-2.301655,0.0
1997,-2.352946,-2.263165,-2.353804,-2.280870,-2.355710,-2.313218,-2.352699,-2.259525,-2.352167,-2.256293,...,-2.253520,-2.311065,-2.258367,-2.324370,-2.252470,-2.310902,-2.234828,-2.335362,-2.292608,0.0
1998,-2.372248,-2.264530,-2.373145,-2.282327,-2.375134,-2.314845,-2.371993,-2.260871,-2.371445,-2.257623,...,-2.254841,-2.328153,-2.259713,-2.342218,-2.253785,-2.328059,-2.236054,-2.353659,-2.294126,0.0


In [ ]:
dls = TabularDataLoaders.from_df(df, path, procs=[], cat_names=[], cont_names=list(df.columns[:-1]), 
                                 y_names=df.columns[-1], y_block=CategoryBlock,
                                 valid_idx=list(range(0, len(df), 10)), bs=256)
learn = tabular_learner(dls, metrics=accuracy)

In [ ]:
learn.lr_find()

# relative to tip of nose
```
bs 64
25, 8e-4 -> 97.4 + wd=1e-2 -> 97.7
bs 256
25, 8e-4, wd=1e-2 -> 98.1-98.4
bs 1024
35, 8e-4, wd=1e-2 -> 97.7
```
# relative to top lip indent

slighly lower accuracy but might give us better results if we care about not misclassifying no expression

```
bs 256
25, 8e-4, wd=1e-2 -> 97.3
25, 8e-4, wd=8e-3 -> 97.8

```

In [ ]:
learn = tabular_learner(dls, metrics=accuracy)
learn.fit_one_cycle(25, 1e-3, wd=1e-2)

In [ ]:
learn.recorder.plot_loss()

In [ ]:
learn.model

In [ ]:
import pickle
class NpModelHelper:
    def __init__(self, path, model_id):
        self.model_id, self.path = model_id, path/f'model_{model_id}.pkl'
    def set_state(self, model):
        self.state_dict, state_dict = {}, learn.model.state_dict()
        for k in state_dict:
            self.state_dict[k] = state_dict[k].detach().cpu().numpy()
        return self
    def save(self):
        # can't set allow_pickle=False with np.savez https://github.com/numpy/numpy/issues/13983
        # so we might as well pickle and keep the dict order
        # TODO: don't need the dict order any more - go back to npz
        with open(self.path, 'wb') as f:
            pickle.dump(self.state_dict, f)
        return self
    def load(self):
        with open(self.path, 'rb') as f:
            self.state_dict = pickle.load(f)
        return self
    def get_state_dict(self, name_prefix):
        if name_prefix is None or name_prefix == '': 
            return self.state_dict
        return {k[len(name_prefix)+1:]:self.state_dict[k] 
                for k in self.state_dict 
                if k.startswith(name_prefix)}

In [ ]:
np_model_helper = NpModelHelper(path, dataset_id)
np_model_helper.set_state(learn.model).save()

In [ ]:
np_model_helper = NpModelHelper(path, dataset_id).load()

In [ ]:
class NpBatchNorm1d:
    # https://github.com/pytorch/pytorch/blob/420b37f3c67950ed93cd8aa7a12e673fcfc5567b/aten/src/ATen/native/Normalization.cpp#L61-L126
    def __init__(self, weight, bias, running_mean, running_var, num_batches_tracked=None):
        self.weight, self.bias = weight, bias
        self.running_mean, self.running_std = running_mean, np.sqrt(running_var + 1e-5)
    def __call__(self, x):
        x = x - self.running_mean
        x = x / self.running_std
        x = x * self.weight
        x = x + self.bias
        return x

In [ ]:
class NpLinear:
    def __init__(self, weight, bias=None):
        self.weight, self.bias = weight.T, bias
    def __call__(self, x):
        x = x @ self.weight
        if self.bias is not None:
            x = x + self.bias
        return x

In [ ]:
class NpReLU:
    def __call__(self, x):
        return np.maximum(x, 0)

In [ ]:
class NpModel:
    def __init__(self, modules):
        self.modules = modules
    def __call__(self, x):
        for module in self.modules:
            x = module(x)
        return x

In [ ]:
np_model = NpModel([
                    NpBatchNorm1d(**np_model_helper.get_state_dict('bn_cont')),
                    NpLinear(**np_model_helper.get_state_dict('layers.0.0')),
                    NpReLU(),
                    NpBatchNorm1d(**np_model_helper.get_state_dict('layers.0.2')),
                    NpLinear(**np_model_helper.get_state_dict('layers.1.0')),
                    NpReLU(),
                    NpBatchNorm1d(**np_model_helper.get_state_dict('layers.1.2')),
                    NpLinear(**np_model_helper.get_state_dict('layers.2.0'))
                   ])

In [ ]:
learn.model

In [ ]:
class_id_to_label = {
        0: "No expression",
        1: "oo",
        2: "ee",
        3: "ah",
        4: "Random Talking"
    }

In [ ]:
confusion_matrix = np.zeros([5,5], dtype=int) # TODO: don't hard code
output = np_model(df[cont_names].to_numpy())
preds = np.argmax(output, axis=1)
targets = df['expression_id'].to_numpy()
for p,t in zip(preds, targets):
    confusion_matrix[t][p]+=1

In [ ]:
confusion_matrix

In [ ]:
# confusion_matrix = np.array([[0,2,1],[1,2,0],[2,1,0]])
fig, ax = plt.subplots(figsize=(9,9))
ax.matshow(confusion_matrix, cmap=plt.cm.Blues, alpha=0.8)
# ax.xaxis.set_ticks_position('bottom') # must be after matshow
for i in range(confusion_matrix.shape[0]):
    for j in range(confusion_matrix.shape[1]):
        ax.text(x=j, y=i,s=confusion_matrix[i, j], va='center', ha='center', size='xx-large')
plt.title('Confusion Matrix', fontsize=14)
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('Actual', fontsize=12)
tick_marks = np.arange(5)
# plt.xticks(tick_marks, self.data.y.classes, rotation=90)
plt.xticks(tick_marks, class_id_to_label.values(), rotation=90)
plt.yticks(tick_marks, class_id_to_label.values(), rotation=0)
plt.show()

In [ ]:
df